In [63]:
from sympy import Ynm, Symbol, diff, simplify, sqrt, conjugate
from sympy import latex
from sympy.vector import CoordSys3D, gradient
from sympy.abc import l, m
import IPython.display as disp

# Helpers

In [64]:
def pprint(x, simplify_expression: bool = True):
    if simplify_expression:
        return disp.display(simplify(x.expand(func=True)))
    return disp.display(x)


def to_latex(x):
    print(
        latex(x)
        .replace("\\mathbf{{phi}_{S}}", "\\phi")
        .replace("\\mathbf{{theta}_{S}}", "\\theta")
        .replace("\\mathbf{{r}_{S}}", "r")
    )

# Coordinate System

In [65]:
S = CoordSys3D(
    "S",
    transformation="spherical",
    variable_names=("r", "theta", "phi"),
    vector_names=("e_r", "e_t", "e_p"),
)


def conjugate_vector(x):
    res = (
        conjugate(x.dot(S.e_r)) * S.e_r
        + conjugate(x.dot(S.e_t)) * S.e_t
        + conjugate(x.dot(S.e_p)) * S.e_p
    )
    return res.subs(
        {
            conjugate(S.theta): S.theta,
            conjugate(S.phi): S.phi,
            conjugate(S.r): S.r,
            conjugate(m): m,
            conjugate(l): l,
        }
    )

# Complex Vector Spherical Harmonics

In [66]:
# These are complex for now.
def Y(l, m):
    return Ynm(l, m, S.theta, S.phi) * S.e_r


def Psi(l, m):
    grad_Y = gradient(Ynm(l, m, S.theta, S.phi))
    return S.r * grad_Y


def Phi(l, m):
    grad_Y = gradient(Ynm(l, m, S.theta, S.phi))
    return (S.r * S.e_r).cross(grad_Y)

In [67]:
pprint(Y(l, m))
pprint(Psi(l, m))
pprint(Phi(l, m))

S.i*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*exp(S.phi*I*m)*assoc_legendre(l, m, cos(S.theta))/(2*sqrt(pi))

((m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*cot(S.theta)*assoc_legendre(l, m, cos(S.theta)) + sqrt((2*l + 1)*gamma(l - m + 1)/((l**2 + l - m**2 - m)*gamma(l + m + 1)))*sqrt(l**2 + l - m**2 - m)*assoc_legendre(l, m + 1, cos(S.theta)))*exp(S.phi*I*m)/(2*sqrt(pi)))*S.e_t + (I*m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*exp(S.phi*I*m)*assoc_legendre(l, m, cos(S.theta))/(2*sqrt(pi)*sin(S.theta)))*S.e_p

(-I*m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*exp(S.phi*I*m)*assoc_legendre(l, m, cos(S.theta))/(2*sqrt(pi)*sin(S.theta)))*S.e_t + ((m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*cot(S.theta)*assoc_legendre(l, m, cos(S.theta)) + sqrt((2*l + 1)*gamma(l - m + 1)/((l**2 + l - m**2 - m)*gamma(l + m + 1)))*sqrt(l**2 + l - m**2 - m)*assoc_legendre(l, m + 1, cos(S.theta)))*exp(S.phi*I*m)/(2*sqrt(pi)))*S.e_p

In [68]:
pprint(conjugate_vector(Y(l, m)))
pprint(conjugate_vector(Psi(l, m)))
pprint(conjugate_vector(Phi(l, m)))

(-1)**(2*m)*S.i*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*exp(-S.phi*I*m)*assoc_legendre(l, m, cos(S.theta))/(2*sqrt(pi))

((-1)**(2*m)*(m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*cot(S.theta)*assoc_legendre(l, m, cos(S.theta)) + sqrt((2*l + 1)*gamma(l - m + 1)/((l**2 + l - m**2 - m)*gamma(l + m + 1)))*conjugate(sqrt((l - m)*(l + m + 1)))*assoc_legendre(l, m + 1, cos(S.theta)))*exp(-S.phi*I*m)/(2*sqrt(pi)))*S.e_t + (-(-1)**(2*m + 1/2)*m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*exp(-S.phi*I*m)*assoc_legendre(l, m, cos(S.theta))/(2*sqrt(pi)*sin(S.theta)))*S.e_p

((-1)**(2*m + 1/2)*m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*exp(-S.phi*I*m)*assoc_legendre(l, m, cos(S.theta))/(2*sqrt(pi)*sin(S.theta)))*S.e_t + ((-1)**(2*m)*(m*sqrt((2*l + 1)*gamma(l - m + 1)/gamma(l + m + 1))*cot(S.theta)*assoc_legendre(l, m, cos(S.theta)) + sqrt((2*l + 1)*gamma(l - m + 1)/((l**2 + l - m**2 - m)*gamma(l + m + 1)))*conjugate(sqrt((l - m)*(l + m + 1)))*assoc_legendre(l, m + 1, cos(S.theta)))*exp(-S.phi*I*m)/(2*sqrt(pi)))*S.e_p

In [69]:
to_latex(Y(l, m).cross(Psi(l, m)).dot(Phi(l, m)))

- \frac{m^{2} \left(Y_{l}^{m}\left(\theta,\phi\right)\right)^{3}}{\sin^{2}{\left(\theta \right)}} + \left(m \cot{\left(\theta \right)} Y_{l}^{m}\left(\theta,\phi\right) + \sqrt{\left(l - m\right) \left(l + m + 1\right)} e^{- \phi i} Y_{l}^{m + 1}\left(\theta,\phi\right)\right)^{2} Y_{l}^{m}\left(\theta,\phi\right)


# Integration

In [70]:
from sympy import pi, sin, cos
from sympy import conjugate
from sympy.vector import ParametricRegion, vector_integrate
from sympy.abc import theta, phi

In [71]:
pprint(Y(1, 0))
pprint(Phi(1, 0))
pprint(Psi(2, 0))

sqrt(3)*S.e_r*cos(S.theta)/(2*sqrt(pi))

-sqrt(3)*S.e_p*sin(S.theta)/(2*sqrt(pi))

-3*sqrt(5)*S.e_t*sin(2*S.theta)/(4*sqrt(pi))

In [85]:
def integrate_on_sphere(func):
    print("Integrand:")
    pprint(func, simplify_expression=True)

    sphere = ParametricRegion(
        (sin(theta) * cos(phi), sin(theta) * sin(phi), cos(theta)),
        (theta, 0, pi),
        (phi, 0, 2 * pi),
    )
    print("Integral over sphere:")
    expr = simplify(func.expand(func=True)).subs({S.theta: theta, S.phi: phi, S.r: 1})
    return vector_integrate(expr, sphere)

# pprint((Phi(1, 1).cross(Phi(1, -1))))
# pprint(conjugate_vector(Psi(1, 0)))

pprint(
    integrate_on_sphere((Phi(1, 1).cross(Phi(1, -1))).dot(conjugate_vector(Y(1, 0)))),
    simplify_expression=True,
)

3*S.e_r*I*cos(S.theta)/(4*pi)

-sqrt(3)*S.e_t*sin(S.theta)/(2*sqrt(pi))

Integrand:


3*sqrt(3)*I*cos(S.theta)**2/(8*pi**(3/2))

Integral over sphere:


sqrt(3)*I/(2*sqrt(pi))